In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/MDE_Capstone/PreReq/

MonoDepth-PyTorch-master      monodepth_resnet18_001.pth
MonoDepth-PyTorch-master.zip


In [ ]:
!mkdir -p models data/models data/output data/test/left/image_02/data/
!cp /content/drive/MyDrive/MDE_Capstone/PreReq/monodepth_resnet18_001.pth data/models/
#!ls -lR /content/

In [ ]:
!cp /content/drive/MyDrive/MDE_Capstone/PreReq/MonoDepth-PyTorch-master/*.py /content

In [ ]:
import main_monodepth_pytorch, loss, transforms, data_loader, models_resnet

In [ ]:
import os
import torch
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
from pathlib import Path

from main_monodepth_pytorch import Model
%reload_ext autoreload
%autoreload 2

In [ ]:
!ls /content/drive/MyDrive/MDE_DS/kitti/
!ls /content/drive/MyDrive/MDE_DS/kitti/train/


train  val
2011_09_26_drive_0001_sync  2011_09_26_drive_0064_sync
2011_09_26_drive_0005_sync  2011_09_26_drive_0070_sync
2011_09_26_drive_0009_sync  2011_09_26_drive_0084_sync
2011_09_26_drive_0011_sync  2011_09_26_drive_0086_sync
2011_09_26_drive_0013_sync  2011_09_26_drive_0087_sync
2011_09_26_drive_0015_sync  2011_09_26_drive_0091_sync
2011_09_26_drive_0017_sync  2011_09_26_drive_0093_sync
2011_09_26_drive_0018_sync  2011_09_26_drive_0095_sync
2011_09_26_drive_0019_sync  2011_09_26_drive_0096_sync
2011_09_26_drive_0022_sync  2011_09_26_drive_0101_sync
2011_09_26_drive_0023_sync  2011_09_26_drive_0104_sync
2011_09_26_drive_0027_sync  2011_09_26_drive_0106_sync
2011_09_26_drive_0028_sync  2011_09_26_drive_0113_sync
2011_09_26_drive_0029_sync  2011_09_26_drive_0117_sync
2011_09_26_drive_0032_sync  2011_09_28_drive_0001_sync
2011_09_26_drive_0035_sync  2011_09_28_drive_0002_sync
2011_09_26_drive_0036_sync  2011_09_29_drive_0004_sync
2011_09_26_drive_0039_sync  2011_09_29_drive_0026_sync

In [ ]:
!ls -l /content/drive/MyDrive/MDE_DS/kitti/train/2011_10_03_drive_0047_sync/image_02/data/*.png | wc -l
!ls -l /content/drive/MyDrive/MDE_DS/kitti/train/2011_10_03_drive_0047_sync/image_03/data/*.png | wc -l


837
837


In [ ]:
KITTI_ROOT = Path("/content/drive/MyDrive/MDE_DS/kitti")
KITTI_VAL, KITTI_TRAIN = KITTI_ROOT / "val", KITTI_ROOT / "train"
#!ls $KITTI_TRAIN $KITTI_VAL

In [ ]:
#Copy files for testing
!cp $KITTI_VAL/2011_09_26_drive_0014_sync/image_02/data/* /content/data/test/left/image_02/data/
!ls $KITTI_VAL/2011_09_26_drive_0014_sync/image_02/data | wc -l

314


In [ ]:
!ls -l /content/data/test/left/image_02/data/ | wc -l 

315


In [ ]:
#!ls -l $KITTI_VAL/2011_09_26_drive_0014_sync/image_02/data

total 31742
-rw------- 1 root root 908046 Jan  9  2012 0000000004.png
-rw------- 1 root root 916900 Jan  9  2012 0000000008.png
-rw------- 1 root root 886090 Jan  9  2012 0000000019.png
-rw------- 1 root root 868121 Jan  9  2012 0000000021.png
-rw------- 1 root root 859420 Jan  9  2012 0000000024.png
-rw------- 1 root root 856396 Jan  9  2012 0000000033.png
-rw------- 1 root root 845296 Jan  9  2012 0000000042.png
-rw------- 1 root root 853400 Jan  9  2012 0000000045.png
-rw------- 1 root root 858366 Jan  9  2012 0000000047.png
-rw------- 1 root root 849762 Jan  9  2012 0000000061.png
-rw------- 1 root root 827945 Jan  9  2012 0000000068.png
-rw------- 1 root root 796086 Jan  9  2012 0000000077.png
-rw------- 1 root root 782705 Jan  9  2012 0000000081.png
-rw------- 1 root root 791470 Jan  9  2012 0000000085.png
-rw------- 1 root root 803190 Jan  9  2012 0000000089.png
-rw------- 1 root root 754376 Jan  9  2012 0000000103.png
-rw------- 1 root root 776447 Jan  9  2012 0000000106.png
-r

## Train

Check if CUDA is available

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.device_count()

1

In [ ]:
torch.cuda.empty_cache()

**Training Dataset(s)**

In [ ]:
dict_parameters = edict({'data_dir':KITTI_TRAIN,
                         'val_data_dir':KITTI_VAL,
                         'model_path':'data/models/monodepth_resnet18_001.pth',
                         'output_directory':'data/output/',
                         'input_height':256,
                         'input_width':512,
                         'model':'resnet18_md',
                         'pretrained':True,
                         'mode':'train',
                         'epochs':200,
                         'learning_rate':1e-4,
                         'batch_size': 8,
                         'adjust_lr':True,
                         'device':'cuda:0',
                         'do_augmentation':True,
                         'augment_parameters':[0.8, 1.2, 0.5, 2.0, 0.8, 1.2],
                         'print_images':False,
                         'print_weights':False,
                         'input_channels': 3,
                         'num_workers': 8,
                         'use_multiple_gpu': False})

In [ ]:
!ls -l /content/drive/MyDrive/MDE_DS/kitti/val/2011_09_26_drive_0014_sync/image_02/data/*.png | wc -l
!ls -l /content/drive/MyDrive/MDE_DS/kitti/val/2011_09_26_drive_0014_sync/image_03/data/*.png | wc -l


314
314


In [ ]:
model = Model(dict_parameters)
#model.load('data/models/monodepth_resnet18_001_last.pth')

Use a dataset with 4400 images
Use a dataset with 37982 images


In [ ]:
!date

Fri Jan 13 04:12:18 UTC 2023


In [ ]:
model.train()

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Val_loss: 3.111454901261763
Epoch: 1 train_loss: 2.402259126356582 val_loss: 2.0572396917776627 time: 8376.465 s
Model_saved
Epoch: 2 train_loss: 2.025853875500539 val_loss: 1.7984904891794378 time: 6438.341 s
Model_saved
Epoch: 3 train_loss: 1.7834488837227211 val_loss: 1.6649380081350154 time: 7153.561 s
Model_saved


In [ ]:
!date

## Test the model

In [ ]:
dict_parameters_test = edict({'data_dir':'data/test',
                              'model_path':'data/models/monodepth_resnet18_001_cpt.pth',
                              'output_directory':'data/output/',
                              'input_height':256,
                              'input_width':512,
                              'model':'resnet18_md',
                              'pretrained':False,
                              'mode':'test',
                              'device':'cuda:0',
                              'input_channels':3,
                              'num_workers':4,
                              'use_multiple_gpu':False})
model_test = Model(dict_parameters_test)

In [ ]:
model_test.test()

In [ ]:
disp = np.load('data/output/disparities_pp.npy')  # Or disparities.npy for output without post-processing
disp.shape

In [ ]:
disp_to_img = skimage.transform.resize(disp[0].squeeze(), [375, 1242], mode='constant')
plt.imshow(disp_to_img, cmap='plasma')

Save a color image

In [ ]:
plt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'_test_output.png'), disp_to_img, cmap='plasma')

Save all test images

In [ ]:
for i in range(disp.shape[0]):
    disp_to_img = skimage.transform.resize(disp[i].squeeze(), [375, 1242], mode='constant')
    plt.imsave(os.path.join(dict_parameters_test.output_directory,
               'pred_'+str(i)+'.png'), disp_to_img, cmap='plasma')

Save a grayscale image

In [ ]:
plt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'_gray.png'), disp_to_img, cmap='gray')

In [ ]:
!ls -l /content/data/output

In [ ]:
!zip 13Dec22_run.zip /content/data/output/*.* /content/data/test/left/image_02/data/*.* /content/data/models/*.*